# Customization

In [1]:
USERNAME = 'joloujo'
TIMEZONE = 'US/Eastern'

## Set up pyLast

In [2]:
from dotenv import load_dotenv
import os
import pylast

# Load environment variables from .env file
load_dotenv()
API_KEY = os.getenv('PYLAST_API_KEY')
API_SECRET = os.getenv('PYLAST_API_SECRET')

# Make sure environment variables were successfully loaded
if API_KEY == None or API_SECRET == None:
    if API_KEY == None:
        print('API Key not found')
    if API_SECRET == None:
        print('API Secret not found')
    print('Exiting')
else:
    network = pylast.LastFMNetwork(API_KEY, API_SECRET)
    network.enable_rate_limit()
    print('Successfully set up pyLast')


Successfully set up pyLast


## Load tracks

In [36]:
from datetime import datetime
from pathlib import Path
import pandas as pd
from pylast import PlayedTrack
import pytz
import time

def track_to_df_entry(track: PlayedTrack):
    return {
        'name': track.track.get_name(),
        'artist': track.track.artist, 
        'album': track.album, 
        'timestamp': int(track.timestamp) if track.timestamp else None, 
        'duration': None,
    }

tracks_path = Path("..") / "data" / "tracks.pkl"
columns = ['name', 'artist', 'album', 'timestamp', 'duration']

tracks: pd.DataFrame

timezone = pytz.timezone(TIMEZONE)

if os.path.exists(tracks_path):
    tracks = pd.read_pickle(tracks_path)
    print('Loaded tracks from file')
else:
    tracks = pd.DataFrame(columns=columns)

user = network.get_user(USERNAME)

time_to = round(time.time())
time_from: int = tracks['timestamp'].max() + 1 if tracks['timestamp'].size > 0 else 0

print(f'Loading songs starting from {datetime.fromtimestamp(time_from, timezone).strftime("%m/%d/%Y %H:%M:%S %Z")}')

while True:
    new_tracks = list(user.get_recent_tracks(limit=500, time_to=time_to, time_from=time_from))

    if len(new_tracks) == 0:
        print('Done fetching tracks')
        break

    print(f'fetched tracks {len(tracks) + 1} to {len(tracks) + len(new_tracks)}')

    last_timestamp = new_tracks[-1].timestamp

    new_tracks_df = pd.DataFrame([track_to_df_entry(track) for track in new_tracks])

    tracks = pd.concat([tracks, new_tracks_df], ignore_index=True)

    if last_timestamp == None: 
        print('Song with no timestamp encountered')
        break

    time_to = int(last_timestamp)

tracks.to_pickle(tracks_path)
print('Saved tracks to file')

print(f'Loaded songs up to {datetime.fromtimestamp(tracks['timestamp'].max(), timezone).strftime("%m/%d/%Y %H:%M:%S %Z")}')



Loaded tracks from file
Loading songs starting from 11/05/2025 00:06:36 EST
Done fetching tracks
Saved tracks to file
Loaded songs up to 11/05/2025 00:06:35 EST
